# Fibonacci Heaps

Realizado por André Reis - fc58192 no âmbito da disciplina de Desenho e Desenvolvimento de Algoritmos 2024/2025

## Introdução

Este trabalho tem como tema principal Fibonacci Heaps. Para introduzir o que se trata isto é necessário explicar o que é uma heap neste contexto.

No contexto de algoritmos, uma heap trata-se de uma estrutura de dados em árvore que dado um node "pai" com um valor k, os seus "filhos" terão sempre um valor <=k (max heap) ou >=k (min heap). Os nodes "filhos" desses "filhos" terão sempre de cumprir a mesma regra que o seu node "pai" cumpre, ou seja, ou a árvore será toda max heap ou será toda min heap. Esta propriedade mantém-se valida independente do tamanho da árvore. [2][4]

Sabendo a definição de heap podemos dizer que uma Fibonacci Heap é uma estrutura de dados que consiste em várias árvores min heap e é usada em operações de priority queue.[1]
Mais à frente irei explicar o seu funcionamento mais em detalhe.

## Motivation and historical background

As Fibonacci Heaps foram criadas em 1984 por Michael Fredman e Robert Tarjan, dois engenheiros informáticos americanos. [1][5][6] Este algoritmo foi originalmente criad0 com o objetivo de melhorar o tempo amortizado do algoritmo de caminho mais curto de Djikstra. [7] Este algoritmo tem também um melhor desempenho quando comparado a binomial heaps em operações como decrese-key e merge e com o mesmo desempenho nas restantes operações. [9][10] 

O nome Fibonacci é dado devido as regras usadas na junção de árvores e corte de nós nas árvores. Estes números coincidem com a sequencia de Fibonacci e dai originar o nome Fibonacci Heaps.[1] Explicarei este promenor mais em detalhe mais à frente.

## Design of the algorithms


### Estrutura
Como já foi referido, Fibonacci Heaps é um conjunto de árvores min heap. Sendo assim a sua estrutura é semelhante a binomial heaps mas bastante mais flexivel sendo até possivel ser um conjunto de árvores de um node só. Isto no entanto não é ideal e eventualmente a árvore é organizada para que se obtenha a complexidade desejada.[1][7]

[7]A sua estrutura contém:
- uma linked list circular com os pointers dos nós de origem de cada árvore
- um conjunto de nós marcados
- pointer para o menor nó de origem

[7]Cada nó contém:
- um pointer para o seu "pai"
- os pointers dos seus "filhos"
- um pointer para os seus "irmãos" da esquerda e direita
- o número de "filhos" que contém
- se está marcado ou não. (um nó é marcado se um dos seus filhos foi cortado, exceto se este for o nó de origem, falado mais a frente nas operações de decrease-key)[1]

### Operações
[1][7]Com esta estrutura é possivel realizar algumas operações em tempo constante tais como: 
- Find-min: encontrar o minimo é constante pois já temos um pointer para o menor nó de origem que será sempre o menor nó de todos
- Merge: esta operação apenas concatena as duas linked lists dos nós de origem das árvores e muda qual é o minimo. Tem um custo constante amortizado
- Insert: igual ao merge mas com uma árvore de um só node. Custo igual ao merge
- Decrease-key: esta operação diminui o valor de um nó. Existem ações diferentes a tomar dependo do estado da árvore, nós marcados, e valores dos nós. Existem 3 situações possiveis:
    1. Caso o nó diminuido continue a ser maior que o seu pai então tudo permanece igual.
    2. Caso o nó diminuido fique menor que o seu pai e o seu pai é um nó não marcado então: marca-se o pai, retira-se o nó diminuido (e desmarca-o) e todos os seus filhos passando a ser um nó de origem de uma árvore.
    3. Caso o nó diminuido fique menor que o seu pai e o seu pai é um nó marcado então: retira-se o nó diminuido (e desmarca-o) e todos os seus filhos passando a ser um nó de origem de uma árvore. O pai sofre a mesma alteração do seu filho, tornando-se também um nó de origem de uma árvore. Este processo é repetido recorsivamente caso o pai desse nó seja também um nó marcado

Existem mais operações no entanto nem todas tem um custo constante amortizado, como é exemplo do Delete que têm um custo de O(log n) ou do extract que junta as árvores numa só.[7]
A operação Extract é a operação que faz o trabalho necessário para que as outras operações tenham um tempo amortizado constante. Assim, esta operação é a mais custosa de todas e é realizada quando ocorre um Delete. A operação Delete é uma junção do Decrease-key e Extract, devido a isso acaba por ter a mesma complexidade do Extract de O(log n).[1][7]

O modo de funcionamento do Extract é o seguinte:
1. Escolhe o menor node como a origem da nova árvore, separando-o dos seus filhos e respetivas árvores originarias.
2. Compara os nós origem restantes, incluindo os filhos do menor nó, em termos do número de filhos que cada um tem
3. Quando deteta dois nós com o mesmo número de filhos junta as duas árvores com o menor desses sendo o pai do outro
4. Repete 3. recursivamente até que todos os nós restantes tenham um número de filhos diferente dos outros.
5. Os nós resultantes de 4. tornam-se filhos do menor nó (escolhido em 1.) resultanto assim de apenas uma árvore

### Fibonacci
A razão pela qual este algoritmo se chamar Fibonacci Heaps deve-se a sua relação com a sequência fibonacci que irá limitar o número de filhos que cada nó tem transformando assim funções que percorrem os filhos dos nós em O(log(n)) em vez de O(n).
Esta relação da-se da seguinte forma:
sendo "x" um nó e "d" o seu numero de filhos, então o número de nós na arvore de root "x" >= F(d+2) sendo F a função Fibonacci.
Esta propriadade deve-se às regras de funcionamento da função Extract_min e limita o crescimento exponencial das árvores na Heap [1][7]

## Implementation in Python 

Neste capitulo irei fazer uma implementação em python deste algoritmo. Farei uma estrutura e de seguida as operações necessárias descritas anteriormente.

Agora é necessário criar os nós das árvores. Estes nós juntamente com as suas funções auxiliares irão atuar como as árvores em si

In [10]:
#class Node representa um node na árvore
class Node:

    def __init__(self, value):
        self.value = value
        self.parent = None
        self.children = None
        self.leftBrother = self
        self.rightBrother = self
        self.numChild = 0
        self.marked = False

    #marcar um node de forma mais legivel
    def mark(self):
        self.marked = True
    
    #desmarcar um node de forma mais legivel
    def unmark(self):
        self.marked = False

    #inserir um outro node como filho do self
    def insert(self, newNode):
        #parent
        newNode.parent = self 

        if(self.children == None):
            self.children = newNode
        else:
            #brothers
            #direita do novo
            newNode.rightBrother = self.children.rightBrother #ligação novo -> velho
            self.children.rightBrother.leftBrother = newNode #ligação velho -> novo
            #esquerda do novo
            newNode.leftBrother = self.children #ligação novo -> velho
            self.children.rightBrother = newNode #ligação velho -> novo

        #nChild
        self.numChild +=1

    #auto-remover-se da linked list de irmãos
    def removeFromBrothers(self):
        self.leftBrother.rightBrother = self.rightBrother
        self.rightBrother.leftBrother = self.leftBrother
        self.leftBrother = self
        self.rightBrother = self
        if(self.parent != None):
            self.parent.numChild -= 1
            self.parent.mark()

    #inserir-se na linked list de irmãos de outro nó
    def insertAsBrother(self, node):
        self.leftBrother = node.leftBrother
        self.rightBrother = node
        node.leftBrother = self
        self.leftBrother.rightBrother = self

    #juntar duas linked lists de nós de irmãos
    def mergeBrothers(self, node):
        last1 = self.leftBrother
        last2 = node.leftBrother
        self.leftBrother = last2
        last2.rightBrother = self
        node.leftBrother = last1
        last1.rightBrother = node

    
    #Test function
    def printNode(self):
        print("SELF: ", self.value)
        print("Parent: ", self.parent)
        print("Left: ", self.leftBrother.value," - right: ", self.rightBrother.value)
        print("Child: ", self.children)
    

Por último implementamos a heap em si. Esta heap conterá um LinkedNode ligado a outras árvores. É necessário também fazer as funções descritas em operações.

In [25]:
#class Heap representa uma fibonacci heap
class Heap:

    def __init__(self, min):
        self.min = min
        self.marked = set()
    
    #inserir um novo nó na heap
    def insert(self, new):
        new.insertAsBrother(self.min)
        if(new.value < self.min.value):
            self.min = new

    #juntar duas heaps (simplesmente juntar as suas duas rootlists)
    def merge(self, newHeap):
        self.min.mergeBrothers(newHeap.min)
        if(self.min.value > newHeap.min.value):
            self.min = newHeap.min
    
    #devolve o menor nó
    def findMin(self):
        return self.min

    #diminui o valor de um determinado nó
    def decrease_key(self, node, newValue):

        #função auxiliar do decrease key, arranca um nó do seu pai e leva-o para a root-list 
        def toRoot(heap, node):
            loop = node.parent.marked
            parent = node.parent
            #remove da lista de irmãos
            node.removeFromBrothers()
            #adiciona a root
            heap.min.insertAsBrother(node) 
            #tirar pai
            node.parent = None
            #unmark
            node.unmark()

            if(loop):
                heap = toRoot(heap, parent)

            return heap
        #muda valor
        node.value = newValue
        if(node.parent != None and node.parent.value > newValue):
            self = toRoot(self, node)

    #devolve e remove o menor nó. Também reestrutura a heap de acordo com as regras de uma fibonacci heap
    def extract_min(self):
        rootList = self.min.rightBrother 
        self.min.removeFromBrothers() # separa o min do resto

        if(self.min.children != None):
            rootList.mergeBrothers(self.min.children) # junta filhos do min com root list original
        new_min = rootList

        #junção de roots de acordo com numero de filhos
        nChildDict = dict()
        while(rootList not in nChildDict):
            rootList.parent = None
            if(rootList.value < new_min.value):
                new_min = rootList
            if(rootList.numChild not in nChildDict.keys()):
                nChildDict[rootList.numChild] = rootList
                rootList = rootList.rightBrother
                break
            else:
                n = rootList.numChild
                other = nChildDict.get(n)
                next = rootList.rightBrother
                if(rootList.value < other.value):
                    other.removeFromBrothers()
                    rootList.insert(other)
                    nChildDict.pop(n)
                    nChildDict[rootList.numChild] = rootList
                else:
                    rootList.removeFromBrothers()
                    other.insert(rootList)
                    nChildDict.pop(n)
                    nChildDict[other.numChild] = other
                rootList = next
        
        #novo min
        old_min = self.min
        self.min = new_min

        #garantir valores do min
        self.min.children = rootList
        self.min.parent = None
        self.min.rightBrother = self.min
        self.min.leftBrother = self.min
        
        
        return old_min

    #apaga um nó tornando-o o menor nó e extraindo-o   
    def delete_node(self, node):
        min_value = self.min.value
        self.decrease_key(node, min_value-1)
        self.extract_min()
        
    #Test function
    def printAll(self):
        def printRec(node):
            node.printNode()
            if(node.children != None):
                printRec(node.children)
            if(node != node.rightBrother):
                brother = node.rightBrother
                while(brother != node):
                    brother.printNode()
                    if(brother.children != None):
                        printRec(brother.children)
                    brother = brother.rightBrother
        
        printRec(self.min)
        



In [26]:
#Codigo de testes
n1 = Node(1)
n2 = Node(2)
n3 = Node(3)
n4 = Node(4)
n5 = Node(5)
n6 = Node(6)
n7 = Node(7)
n8 = Node(8)
n9 = Node(9)
n10 = Node(10)

heap = Heap(n1)
heap.insert(n2)
heap.insert(n3)
heap.insert(n4)
heap.insert(n5)
heap.insert(n6)
heap.insert(n7)
heap.insert(n8)
heap.insert(n9)
heap.insert(n10)

heap.delete_node(n5)

heap.printAll()

SELF:  2
Parent:  None
Left:  2  - right:  2
Child:  <__main__.Node object at 0x000002158B761650>
SELF:  3
Parent:  None
Left:  2  - right:  4
Child:  None
SELF:  4
Parent:  None
Left:  3  - right:  0
Child:  None
SELF:  0
Parent:  None
Left:  4  - right:  6
Child:  None
SELF:  6
Parent:  None
Left:  0  - right:  7
Child:  None
SELF:  7
Parent:  None
Left:  6  - right:  8
Child:  None
SELF:  8
Parent:  None
Left:  7  - right:  9
Child:  None
SELF:  9
Parent:  None
Left:  8  - right:  10
Child:  None
SELF:  10
Parent:  None
Left:  9  - right:  2
Child:  None
SELF:  2
Parent:  None
Left:  2  - right:  2
Child:  <__main__.Node object at 0x000002158B761650>
SELF:  3
Parent:  None
Left:  2  - right:  4
Child:  None
SELF:  4
Parent:  None
Left:  3  - right:  0
Child:  None
SELF:  0
Parent:  None
Left:  4  - right:  6
Child:  None
SELF:  6
Parent:  None
Left:  0  - right:  7
Child:  None
SELF:  7
Parent:  None
Left:  6  - right:  8
Child:  None
SELF:  8
Parent:  None
Left:  7  - right:  9
Chi

RecursionError: maximum recursion depth exceeded

## Análise de complexidade

### Classe Node

A classe Node serve como base para o algoritmo num todo. Esta contém as funções:
- mark
- unmark
- insert
- removeFromBrothers
- insertAsBrother
- mergeBrothers

Todas estas funções tem complexidade O(1). Vejamos agora cada uma individualmente.

#### mark
Simplesmente muda uma váriavel, sendo assim O(1).

#### unmark
Igual a "mark", simplesmente muda uma váriavel, sendo assim O(1).

#### insert
Não contém qualquer tipo de loop e limita-se a mudar os vários parametros de diversos nós. O número de nós e parametros alterados é sempre constante e devido a isso esta função é também O(1).

#### removeFromBrothers
Limita-se a alterar as variaveis de pointers de irmãos de 3 nós. Não contém qualquer loop e todas as chamadas da função terão o mesmo tempo. Devido a isso é O(1).

#### insertAsBrother
Esta função faz o oposto de "removeFromBrothers" da mesma forma. Não contém qualquer tipo de loop e é O(1).

#### mergeBrothers
Semelhante às duas funções anteriores, apenas muda sempre as mesmas variaveis e não contém qualquer tipo de loop. É por isso também O(1).


### Classe Heap
A classe Heap contém as funções principais esperadas do algoritmo. Estas são:
- insert
- merge
- findMin
- decrease_key, esta função contém a função "toRoot" 
- extract_min
- delete_node 

A maioria tem complexidade O(1) mas não todas. Vejamos agora a cada uma individualmente.

#### insert
Esta função limita-se a adicionar um node à rootList. Para tal, esta chama a função "insertAsBrothers" da classe Node. Como visto anteriormente esta função tem complexidade O(1). Devido a isso, a função "insert" tem também complexidade O(1).

#### merge
Semelhantemente à função anterior, o merge limita-se a chamar uma função da classe Node, sendo esta a função "mergeBrothers". Esta função é também O(1) o que resulta na função merge ser também O(1).

#### findMin
Esta função é um simples "getter" sendo por isso O(1).

#### decrease-key
Decrease-key é uma função dividida em duas partes. A sua parte final é uma simples atribuição de valores sendo por isso O(1). Por outro lado, a primeira parte é uma chamada à função "toRoot" contida dentro de si. Devido a isto, a complexidade desta função é depende da complexidade de "toRoot"

#####  toRoot
Esta função está também dividida em duas partes. A primeira parte é uma simples atribuição de valores O(1). A segunda parte da função é uma recursão. Esta recursão, no entanto, só irá ocorrer caso o pai do nó em questão esteja marcado. Para que um nó seja marcado será necessário que este perca um filho, e para tal é necessário que se chame a função em questão. Um nó que foi cortado e voltou à root perderá a sua marca. Devido a este facto, todas as chamadas desta função, com exceção da primeira, irão marcar um nó e desmarcar outro. Devido a isto, a sua complexidade irá se manter constante (O(1)).[7]

### extract_min
Esta função é a mais complexa de todo o algoritmo e por isso está dividida em diversas partes.
Numa primeira parte é retirado o min e os seus filhos são adicionados à rootList. Esta parte utiliza o "findMin" e o "mergeBrothers" ambas funções da classe Node e ambas com complexidade O(1). Devido a isso, esta primeira parte tem uma complexidade de O(1).
Na segunda parte é necessário percorrer os nós todos pertencentes à rootList e junta-los de acordo com o número de filhos que cada um tem. Para guardar quantos "filhos" cada nó percorrido tem em comparação com outros é usado um dicionário, devido a isso a comparação de nós tem uma complexidade neglegenciavel quando comparado com a complexidade de ter de percorrer todos os nós da rootList. Com isto, esta parte teria complexidade O(n) onde n é o número de nós na rootList. No entanto, devido às regras relativas ao número de filhos dos nós aplicadas por esta função relacionadas com o crescimento da árvore e a sequência de fibonacci, então esta secção acaba por ter uma complexidade de O(log(n)).
A ultima parte da função destina-se a uma simples mudaça de valores com tempo constante O(1).
Juntando as 3 partes da função conclui-se que esta tem uma complexidade O(log(n))


## Exercicio relacionado

## Referencias:

1. https://en.wikipedia.org/wiki/Fibonacci_heap 
2. https://en.wikipedia.org/wiki/Heap_(data_structure) 
3. https://en.wikipedia.org/wiki/Priority_queue 
4. https://www.geeksforgeeks.org/heap-data-structure/ 
5. https://en.wikipedia.org/wiki/Michael_Fredman
6. https://en.wikipedia.org/wiki/Robert_Tarjan
7. https://www.cs.princeton.edu/~wayne/kleinberg-tardos/pdf/FibonacciHeaps.pdf
8. https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm 
9. https://en.wikipedia.org/wiki/Binomial_heap 
10. https://www.cs.cmu.edu/afs/cs/academic/class/15750-s17/ScribeNotes/lecture3.pdf 
11. https://github.com/danielborowski/fibonacci-heap-python/blob/master/fib-heap.py
12. https://dl.acm.org/doi/pdf/10.1145/2213977.2214082 